# MultiHandler

Multi-task synchronized acquisition with hardware triggers.

In [ ]:
# Check nidaqmx availability
try:
    import nidaqmx
except ImportError:
    raise RuntimeError(
        "nidaqmx is not installed. Install with: pip install nidaqmx"
    )

In [ ]:
import numpy as np
from nidaqwrapper import MultiHandler, AITask, AOTask, list_devices

# List connected hardware
devices = list_devices()
print(f"Found {len(devices)} device(s):")
for i, dev in enumerate(devices):
    print(f"  Device {i}: {dev['name']} ({dev['product_type']})")

## Basic Multi-Task

Synchronize multiple AITask objects with hardware triggers.

In [ ]:
# Edit device_ind to match your hardware
device_ind = 0

# Create multiple input tasks
task1 = AITask(
    task_name='accel_task',
    sample_rate=10000,
    device_ind=device_ind,
    samples_per_ch=5000,  # FINITE mode for hardware trigger
    clock_source='OnboardClock',
    trigger_source='/Dev1/PFI0',  # Edit to match your trigger source
    trigger_edge='RISING'
)
task1.add_channel(
    channel_name='accel_x',
    channel_ind=0,
    max_val=10.0,
    min_val=-10.0,
    units='V'
)

task2 = AITask(
    task_name='force_task',
    sample_rate=10000,
    device_ind=device_ind,
    samples_per_ch=5000,
    clock_source='OnboardClock',
    trigger_source='/Dev1/PFI0',
    trigger_edge='RISING'
)
task2.add_channel(
    channel_name='force_z',
    channel_ind=1,
    max_val=100.0,
    min_val=-100.0,
    units='V'
)

# Configure and connect
multi = MultiHandler()
success = multi.configure(input_tasks=[task1, task2])
print(f"Configuration successful: {success}")

if success:
    connected = multi.connect()
    print(f"Connected: {connected}")
    
    if connected:
        # Wait for trigger, then acquire
        print("Waiting for hardware trigger...")
        data = multi.acquire()
        
        # Data structure: {task_name: {channel_name: ndarray}}
        print(f"\nAcquired data from {len(data)} tasks:")
        for task_name, channels in data.items():
            print(f"  {task_name}:")
            for ch_name, ch_data in channels.items():
                print(f"    {ch_name}: {ch_data.shape}")
        
        multi.disconnect()

## Input + Output

Synchronized input and output tasks.

In [ ]:
device_ind = 0

# Input task
ai = AITask(
    task_name='sync_input',
    sample_rate=10000,
    device_ind=device_ind,
    samples_per_ch=10000,
    clock_source='OnboardClock',
    trigger_source='/Dev1/PFI0',
    trigger_edge='RISING'
)
ai.add_channel('sensor', channel_ind=0, max_val=10.0, min_val=-10.0, units='V')

# Output task
ao = AOTask(
    task_name='sync_output',
    sample_rate=10000,
    device_ind=device_ind,
    samples_per_ch=10000,
    clock_source='OnboardClock',
    trigger_source='/Dev1/PFI0',
    trigger_edge='RISING'
)
ao.add_channel('stimulus', channel_ind=0, max_val=10.0, min_val=-10.0)

# Configure with both input and output
multi = MultiHandler()
multi.configure(input_tasks=[ai], output_tasks=[ao])
multi.connect()

print(f"Configured {len(multi.input_tasks)} input and {len(multi.output_tasks)} output tasks")
print(f"Trigger type: {multi.trigger_type}")

multi.disconnect()

## Validation

MultiHandler validates task compatibility during configure().

In [ ]:
print("MultiHandler validation checks:")
print("\n1. Type validation - all tasks must be AITask, AOTask, nidaqmx.Task, or str")
print("2. Task resolution - str task names loaded from NI MAX")
print("3. Validity check - tasks must be open with at least one channel")
print("4. Sample rate consistency - all tasks in a group must share the same rate")
print("5. Timing validation - clock source and timing config must match")
print("6. Trigger consistency - all tasks must use the same trigger type/source")
print("7. Acquisition mode - FINITE for hardware triggers, CONTINUOUS for software")
print("\nIf any validation fails, configure() returns False.")

## Context Manager or close()

Proper cleanup pattern for multi-task handlers.

In [ ]:
# Manual cleanup with disconnect()
multi = MultiHandler()
multi.configure(input_tasks=[task1, task2])
multi.connect()

# ... do work ...

multi.disconnect()
print("Disconnected and cleaned up.")

In [ ]:
# Alternative: use disconnect() in a try/finally
multi = MultiHandler()
try:
    multi.configure(input_tasks=[task1, task2])
    multi.connect()
    # ... do work ...
finally:
    multi.disconnect()
    print("Cleanup complete.")